In [1]:
'''Here we reorganize a resnet variant that was trained on RGGB  images'''

'Here we reorganize a resnet variant that was trained on RGGB  images'

In [3]:
import os
import sys
import numpy as np
sys.path.insert(0, "/home/bnapp/arivkindNet/wisdrc/")
from parse_commandline import parse_commandline
from utils.imagenet_dataset import get_dataset
from utils.unprocess import *
from utils.image_utils import *
from utils.image_processing import rescale
from utils.split_keras_model import split_model
from models.student_models import full_size_rggb_fe
import tensorflow as tf
import matplotlib.pyplot as plt

In [4]:
import pickle as pkl

In [5]:
parser = parse_commandline(return_parser=True)
config = parser.parse_args([])
config = vars(config)
parameters = config

In [6]:
rggb_resnet = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 4),
                                                      include_top=True,
                                                      weights=None)

/usr/local/lib/python3.8/dist-packages/keras/applications/imagenet_utils.py:331: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  warnings.warn('This model usually expects 1 or 3 input channels. '
2023-01-03 21:55:35.774065: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1917] Ignoring visible gpu device (device: 1, name: NVIDIA GeForce GT 730, pci bus id: 0000:65:00.0, compute capability: 3.5) with core count: 2. The minimum required count is 8. You can adjust this requirement with the env var TF_MIN_GPU_MULTIPROCESSOR_COUNT.
2023-01-03 21:55:36.183893: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10247 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:17:00.0, compute capability: 6.1


In [7]:
rggb_resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 4) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 4)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 12608       conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [9]:
# val_generator_baseline = get_dataset(parameters['dataset_dir'], 'validation', batch_size, preprocessing=parameters['preprocessing'])


In [10]:
fe_model = full_size_rggb_fe(res=448)

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [11]:
fe_model.summary()

Model: "student_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1, 224, 224, 4)]  0         
_________________________________________________________________
reshape (Reshape)            (None, 224, 224, 4)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 64)      12608     
_________________________________________________________________
fe_be_model (Functional)     (None, 56, 56, 64)        256       
Total params: 12,864
Trainable params: 12,736
Non-trainable params: 128
_________________________________________________________________


In [12]:
fe_model.load_weights('../../drc_saves/saved_models/noname_j_t1672583825/final_weights/fe_final_weights/fe_final_weights')

In [13]:
be_model = keras.models.load_model('../../drc_saves/saved_models/noname_j_t1672583825/final_weights/decoder_final_model')

In [14]:
be_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_top (InputLayer)          [(None, 56, 56, 64)] 0                                            
__________________________________________________________________________________________________
conv2_block1_1_conv (Conv2D)    (None, 56, 56, 64)   4160        input_top[0][0]                  
__________________________________________________________________________________________________
conv2_block1_1_bn (BatchNormali (None, 56, 56, 64)   256         conv2_block1_1_conv[0][0]        
__________________________________________________________________________________________________
conv2_block1_1_relu (Activation (None, 56, 56, 64)   0           conv2_block1_1_bn[0][0]          
______________________________________________________________________________________________

In [15]:
fe_model.layers[-1].summary()

Model: "fe_be_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_top (InputLayer)       [(None, 112, 112, 64)]    0         
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 64)      256       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 64)      0         
_________________________________________________________________
pool1_pad (ZeroPadding2D)    (None, 114, 114, 64)      0         
_________________________________________________________________
pool1_pool (MaxPooling2D)    (None, 56, 56, 64)        0         
Total params: 256
Trainable params: 128
Non-trainable params: 128
_________________________________________________________________


In [17]:
fe_model.get_layer('conv2d')._name = 'conv1_conv'

In [18]:
for layer in rggb_resnet.layers[2:]:
    try:
        new_layer = fe_model.get_layer(layer.name)
    except:
        try: 
            new_layer = fe_model.get_layer('fe_be_model').get_layer(layer.name)
        except:
            new_layer = be_model.get_layer(layer.name)
    layer.set_weights(new_layer.get_weights())
    

        
    


In [19]:
rggb_resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 4) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 4)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 12608       conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [20]:
rggb_resnet.save('rggb_resnet50_rev01')

2023-01-03 21:57:08.807381: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: rggb_resnet50_rev01/assets


/usr/local/lib/python3.8/dist-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
